In [1]:
import duckdb
from typing import Any
import json
import geopandas as gpd
from osgeo import ogr

In [3]:
con = duckdb.connect()
print(con.execute('SELECT 42').fetchall())

[(42,)]


In [4]:
# To install an extension (this is usually done only once)
try:
    con.install_extension('httpfs')
    con.install_extension('spatial')
    con.load_extension('httpfs')
    con.load_extension('spatial')
    con.execute("SET s3_region='us-west-2'")
except Exception as e:
    print(f"Failed to install extension: {e}")

In [5]:
read_query = '''
SELECT
    type,
    subType,
    localityType,
    adminLevel,
    isoCountryCodeAlpha2,
    JSON(names) AS names,
    JSON(sources) AS sources,
    ST_GeomFromWkb(geometry) AS geometry
FROM read_parquet(
    's3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=admins/type=*/*',
    filename=true,
    hive_partitioning=1
)
WHERE adminLevel = 2
    AND ST_GeometryType(ST_GeomFromWkb(geometry)) IN ('POLYGON','MULTIPOLYGON')
'''

In [6]:
copy_query = f'''
COPY ({read_query}) 
TO 'countries.gpkg' 
WITH (FORMAT GDAL, DRIVER 'GPKG');
'''

In [7]:
con.execute(copy_query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))